In [56]:
import pandas as pd
import geonamescache
import re

In [57]:
path = "./data/headlines.txt"
f = open(path, 'r')
headlines = f.read()
headlines_list = headlines.splitlines()
f.close()

# headlines_list = ['Zika Outbreak Hits Miami', 'Could Zika Reach New York City?', 'First Case of Zika in Miami Beach', 'Mystery Virus Spreads in Recife, Brazil',]

In [58]:
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
countries_list = {}
for key, value in countries.items():
    countries_list[key] = value["name"]

# countries_list = {'AD': 'Andorra', 'AE': 'United Arab Emirates'...}

In [59]:
cities = gc.get_cities()
cities_list = {}
for value in cities.values():
    cities_list[value["name"]] = value["countrycode"]

# cities_list = {'Andorra la Vella': 'AD', 'Umm Al Quwain City': 'AE'...}

In [77]:
# To save time while iterating over cities join them with "|" by 100 in group

cities_list_reformatted = []
part_size = 100
start_index = 0
end_index = part_size

index = 0
city_part_list = []
for city_name in cities_list:
    if index < part_size:
        index += 1
    else: 
        cities_list_reformatted.append(city_part_list)
        city_part_list = []
        index = 1
    city_part_list.append(city_name)
if (len(city_part_list)): 
    cities_list_reformatted.append(city_part_list)

# Sort each sublist to put longer words at the beginnig in order regex find them first (eg. "Miami Beach|Miami": otherwise regex find "Miami" and will ignore "Miami Beach")
for cl in cities_list_reformatted:
    cl.sort(key=lambda city: len(city), reverse=True)

cities_list_reformatted = ['|'.join(y) for y in cities_list_reformatted]

# cities_list_reformatted = ['Markaz-e Woluswalī-ye Āchīn|Reef Al Fujairah City|Ras Al Khaimah City|... , ..|..|..]

In [73]:
extracted_places_list = []
for headline in headlines_list:
    hl_with_places = {'headline': headline}
    
    # extract countries     
    for code, country in countries_list.items():
        pattern = '(^|[^a-zA-Z-])(' + country + ')($|[^a-zA-Z-])'
        s = re.search(pattern, headline)
        if s:
            if (not 'country' in hl_with_places):
                hl_with_places['country'] = []
            hl_with_places['country'].append(country)    

    # extract cities
    for cities in cities_list_reformatted:
        pattern = '(^|[^a-zA-Z-])(' + cities + ')($|[^a-zA-Z-])'
        s = re.search(pattern, headline)
        if s:
            if (not 'city' in hl_with_places):
                hl_with_places['city'] = []
            hl_with_places['city'].append(s.group(2))    
    
    extracted_places_list.append(hl_with_places)
            
extracted_places_df = pd.DataFrame(extracted_places_list)
extracted_places_df.head()


,headline,city,country
0,Zika Outbreak Hits Miami,[Miami],NaN
1,Could Zika Reach New York City?,"[York, New York City]",NaN
2,First Case of Zika in Miami Beach,[Miami Beach],NaN
3,"Mystery Virus Spreads in Recife, Brazil",[Recife],[Brazil]
4,Dallas man comes down with case of Zika,[Dallas],NaN


In [74]:
def remove_substrings(city_list):
    i = 0
    while i < len(city_list):
        j = i + 1
        while j < len(city_list):
            pattern = '(^|[^a-zA-Z-])(' + city_list[i] + ')($|[^a-zA-Z-])'
            if (re.search(pattern, city_list[j])):
                del city_list[i]
                break
            else:
                j += 1
        if (j == len(city_list)):
            i += 1    
    return city_list

In [75]:
# remove substrings from city column
for index, row in extracted_places_df.iterrows():
    if (isinstance(row['city'], list) and len(row['city']) > 1):
        row['city'].sort(key=lambda city: len(city))
        remove_substrings(row['city'])

In [76]:
extracted_places_df.head()

,headline,city,country
0,Zika Outbreak Hits Miami,[Miami],NaN
1,Could Zika Reach New York City?,[New York City],NaN
2,First Case of Zika in Miami Beach,[Miami Beach],NaN
3,"Mystery Virus Spreads in Recife, Brazil",[Recife],[Brazil]
4,Dallas man comes down with case of Zika,[Dallas],NaN
